In [176]:
import pandas as pd
import numpy as np

In [177]:
universes = pd.read_csv('./Data/univ_h.csv')
universes = universes.T
universes.columns = universes.iloc[0]
universes = universes.drop(universes.index[0])

In [178]:
adj_close = pd.read_csv('./Data/adjusted.csv', header=None)
adj_close.iloc[0, :] = [''.join(index.split()) for index in adj_close.iloc[0, :]]
adj_close.columns = adj_close.iloc[0, :]
adj_close = adj_close.drop(0)
adj_close['Date'] = pd.to_datetime(adj_close['Date'], format='%Y%m%d')

/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,2

In [179]:
adj_price = pd.DataFrame()

for year in universes.columns:
    df_year = adj_close[adj_close['Date'].dt.year == year].copy()
    ticker = universes[year].replace(0, np.nan).dropna().index
    df_year = df_year.loc[:, ['Date'] + list(ticker)]
    df_year.index = df_year['Date']
    df_year.drop('Date', axis=1, inplace=True)

    adj_price = pd.concat([adj_price, df_year]).sort_index()

adj_price = adj_price.apply(lambda x: pd.to_numeric(x))

In [180]:
daily_returns = adj_price.apply(lambda x: np.log(x / x.shift(1))).fillna(0)

volatility = daily_returns[1:].rolling(window=21).std()
volatility = volatility.applymap(lambda x: 0.005 if x < 0.005 else x)

five_days_returns = adj_price.apply(lambda x: np.log(x / x.shift(5))).fillna(0)

std_returns =  (five_days_returns / volatility).fillna(0)

2 (d)

In [181]:
def get_idt_returns(path, returns):
    idt_code = pd.read_csv(path, index_col=0, header=None)
    idt_code.columns = ['GIC_Code']
    idt_code['GIC_Industry'] = idt_code['GIC_Code'].astype(str).str[:6]
    code = idt_code['GIC_Industry'].unique()[1:]

    R_industry = returns.join(idt_code['GIC_Industry'])

    industry_returns = R_industry.groupby('GIC_Industry').mean().T

    returns_with_industry = R_industry.T.apply(
        lambda column: column[:-1] - industry_returns.loc[:, column[-1]] if column[-1] in code else column[:-1]
    )
    return returns_with_industry

path = './Data/tickers.csv'
std_returns_with_industry = get_idt_returns(path, std_returns.T)
d_returns_with_industry = get_idt_returns(path, daily_returns.T)

### 3

In [182]:
'''
import statsmodels.api as sm

t1_returns = d_returns_with_industry.shift(-1).fillna(0)

reg_coefficient = pd.DataFrame(index=std_returns_with_industry.index, columns=['beta', 'Rsquared'])

for i in range(len(std_returns_with_industry)):
    X = std_returns_with_industry.iloc[i, :]
    y = t1_returns.iloc[i, :]

    X = pd.to_numeric(X, errors='coerce')
    y = pd.to_numeric(y, errors='coerce')

    # X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()

    # print(model.summary())

    beta = model.params[0]
    r_squared = model.rsquared

    reg_coefficient.iloc[i, 0] = beta
    reg_coefficient.iloc[i, 1] = r_squared

reg_coefficient = reg_coefficient.loc['20050103':, :].dropna()
'''

"\nimport statsmodels.api as sm\n\nt1_returns = d_returns_with_industry.shift(-1).fillna(0)\n\nreg_coefficient = pd.DataFrame(index=std_returns_with_industry.index, columns=['beta', 'Rsquared'])\n\nfor i in range(len(std_returns_with_industry)):\n    X = std_returns_with_industry.iloc[i, :]\n    y = t1_returns.iloc[i, :]\n\n    X = pd.to_numeric(X, errors='coerce')\n    y = pd.to_numeric(y, errors='coerce')\n\n    # X = sm.add_constant(X)\n    model = sm.OLS(y, X).fit()\n\n    # print(model.summary())\n\n    beta = model.params[0]\n    r_squared = model.rsquared\n\n    reg_coefficient.iloc[i, 0] = beta\n    reg_coefficient.iloc[i, 1] = r_squared\n\nreg_coefficient = reg_coefficient.loc['20050103':, :].dropna()\n"

In [183]:
def calculate_beta_and_r_squared(R, v):
    """
    Calculate beta(t) and R^2(t) based on the given formulas.

    Parameters:
    R (numpy.ndarray): Array of returns R_i(t+1)
    v (numpy.ndarray): Array of values v_i(t)

    Returns:
    beta (float): The calculated beta(t)
    r_squared (float): The calculated R^2(t)
    """
    # Ensure R and v are numpy arrays
    R = np.array(R)
    v = np.array(v)

    # Calculate beta(t)
    numerator_beta = np.sum(R * v)
    denominator_beta = np.sum(v * v)
    beta = numerator_beta / denominator_beta

    # Calculate residuals
    epsilon = R - beta * v

    # Calculate R^2(t)
    numerator_r_squared = np.sum(epsilon ** 2)
    denominator_r_squared = np.sum(R ** 2)
    r_squared = 1 - (numerator_r_squared / denominator_r_squared)

    return beta, r_squared

In [184]:
std_returns_with_industry.index = pd.to_datetime(std_returns_with_industry.index)
d_returns_with_industry.index = pd.to_datetime(d_returns_with_industry.index)

df_X = std_returns_with_industry.loc['2005-01-03':, :]
df_Y = d_returns_with_industry.shift(-1).loc['2005-01-03':, :]

df_reg_coefficient = pd.DataFrame(index=df_X.index, columns=['beta', 'Rsquared'])

for i in range(len(df_X)):
    v = df_X.iloc[i, :]
    R = df_Y.iloc[i, :]

    beta, r_squared = calculate_beta_and_r_squared(R, v)

    df_reg_coefficient.iloc[i, 0] = beta
    df_reg_coefficient.iloc[i, 1] = r_squared

df_reg_coefficient

,beta,Rsquared
2005-01-03,-0.000495,0.001165
2005-01-04,0.002051,0.071769
2005-01-05,-0.000887,0.027593
2005-01-06,0.000385,0.008437
2005-01-07,-0.00047,0.018504
...,...,...
2024-12-24,0.000054,0.000146
2024-12-26,-0.001425,0.105594
2024-12-27,-0.001663,0.05086
2024-12-30,0.000115,0.000307


### 4

In [185]:
df_reg_coefficient['year'] = df_reg_coefficient.index.to_period('Y')
avg_beta = df_reg_coefficient.groupby('year').mean().loc[:, 'beta']

num_beta = df_reg_coefficient.groupby('year').count().loc[:, 'beta']

std_beta = df_reg_coefficient.groupby('year').std().loc[:, 'beta']

df_t_stat = []
for i in range(len(avg_beta)):
    t_stat = (np.sqrt(num_beta[i]) * avg_beta[i]) / std_beta[i]
    df_t_stat.append(t_stat)

results = pd.DataFrame({'beta': avg_beta, 't_stats': df_t_stat}, index=df_reg_coefficient['year'].unique())
results

,beta,t_stats
2005,-0.000077,-0.975251
2006,-0.000102,-1.306873
2007,-0.000158,-1.276923
2008,-0.000229,-0.466692
2009,0.000004,0.012309
2010,-0.000045,-0.271235
2011,-0.000311,-1.248189
2012,-0.000062,-0.716553
2013,-0.000089,-1.241593
2014,0.000006,0.090565


### 5

In [186]:
def get_ranked(x):
    N = len(x)
    rank = x.rank(ascending=False)
    ranked = (N + 1 - 2*rank) / (N - 1)
    return ranked

ranked_std_R = std_returns.apply(lambda x: get_ranked(x), axis=1)
ranked_std_R_industry = get_idt_returns(path, ranked_std_R.T)

In [188]:
ranked_std_R_industry.index = pd.to_datetime(ranked_std_R_industry.index)
ranked_std_R_industry_df = ranked_std_R_industry.loc['20050103':, :]
t1_returns = d_returns_with_industry.shift(-1).loc['20050103':, :]
reg_coefficient_df = pd.DataFrame(index=ranked_std_R_industry_df.index, columns=['beta', 'Rsquared'])

for i in range(len(ranked_std_R_industry_df)):
    v = ranked_std_R_industry_df.iloc[i, :]
    R = t1_returns.iloc[i, :]

    beta, r_squared = calculate_beta_and_r_squared(R, v)

    reg_coefficient_df.iloc[i, 0] = beta
    reg_coefficient_df.iloc[i, 1] = r_squared

reg_coefficient_df

,beta,Rsquared
2005-01-03,-0.000954,0.001227
2005-01-04,0.004903,0.050748
2005-01-05,-0.002916,0.022726
2005-01-06,0.001537,0.009568
2005-01-07,-0.001994,0.02077
...,...,...
2024-12-24,0.000076,0.000063
2024-12-26,-0.003304,0.113626
2024-12-27,-0.003718,0.079368
2024-12-30,-0.000096,0.000095


In [189]:
reg_coefficient_df['year'] = reg_coefficient_df.index.to_period('Y')
avg_beta = reg_coefficient_df.groupby('year').mean().loc[:, 'beta']

num_beta = reg_coefficient_df.groupby('year').count().loc[:, 'beta']

std_beta = reg_coefficient_df.groupby('year').std().loc[:, 'beta']

df_t_stat = []
for i in range(len(avg_beta)):
    t_stat = (np.sqrt(num_beta[i]) * avg_beta[i]) / std_beta[i]
    df_t_stat.append(t_stat)

results = pd.DataFrame({'beta': avg_beta, 't_stats': df_t_stat}, index=reg_coefficient_df['year'].unique())
results

,beta,t_stats
2005,-0.000236,-1.084922
2006,-0.000355,-1.698890
2007,-0.000374,-1.186144
2008,-0.000898,-0.763922
2009,0.000192,0.256754
2010,-0.000172,-0.409168
2011,-0.000646,-1.089363
2012,-0.000311,-1.243383
2013,-0.000282,-1.445951
2014,-0.000044,-0.219581


#### Part B

In [169]:
betas = pd.DataFrame(avg_beta.shift(1).dropna(), columns=['beta'])
betas.index = betas.index.year

returns = std_returns_with_industry.loc['2006-01-03':, :].copy()
returns.index = returns.index.year

returns = returns.join(betas)
exp_returns = returns.apply(lambda x: x * returns['beta']).iloc[:, :-1]

In [170]:
# Define the function to create long-short portfolio
def Simulated_long_short_portfolio(exp_returns, daily_returns, top_percent=0.2, bottom_percent=0.2):
    long_short_portfolio = pd.DataFrame(index=exp_returns.index, columns=['Long', 'Short', 'Portfolio'])

    for date in exp_returns.index[:-1]:
        df_returns = exp_returns.loc[date, :].dropna()
        num_stocks = len(df_returns)
        num_long = int(num_stocks * top_percent)
        num_short = int(num_stocks * bottom_percent)

        top_stocks = df_returns[df_returns.rank(ascending=False) >= num_stocks - num_long].index.to_list()
        bottom_stocks = df_returns[df_returns.rank(ascending=False) <= num_short].index.to_list()

        next_day = long_short_portfolio.index[exp_returns.index.get_loc(date) + 1]
        long_return = daily_returns.loc[next_day, top_stocks].mean()
        short_return = daily_returns.loc[next_day, bottom_stocks].mean()

        long_short_portfolio.loc[next_day, 'Long'] = long_return
        long_short_portfolio.loc[next_day, 'Short'] = short_return
        long_short_portfolio.loc[next_day, 'Portfolio'] = long_return - short_return

    return long_short_portfolio

# Create the long-short portfolio
exp_returns.index = std_returns_with_industry.loc['2006-01-03':, :].index
long_short_portfolio = Simulated_long_short_portfolio(exp_returns, daily_returns)

# Display the long-short portfolio
long_short_portfolio

,Long,Short,Portfolio
2006-01-03,NaN,NaN,NaN
2006-01-04,0.003718,0.006655,-0.002937
2006-01-05,-0.001865,0.000555,-0.00242
2006-01-06,0.00965,0.005789,0.003861
2006-01-09,0.006199,0.001234,0.004965
...,...,...,...
2024-12-24,0.00243,0.008301,-0.005871
2024-12-26,0.001346,0.001175,0.00017
2024-12-27,-0.007083,-0.002741,-0.004342
2024-12-30,-0.008585,-0.004499,-0.004086


In [171]:
# Calculate annual return and annualized return volatility
annual_returns = long_short_portfolio['Portfolio'].groupby(long_short_portfolio.index.year).sum()
annual_volatility = long_short_portfolio['Portfolio'].groupby(long_short_portfolio.index.year).std() * np.sqrt(252)

# Combine the results into a DataFrame
annual_performance = pd.DataFrame({
    'Annual Return': annual_returns,
    'Annualized Volatility': annual_volatility
})

annual_performance

,Annual Return,Annualized Volatility
2006,-0.126113,0.066564
2007,-0.144599,0.101674
2008,-0.308981,0.379326
2009,0.062507,0.242195
2010,0.064228,0.137949
2011,-0.218834,0.199909
2012,-0.10003,0.084154
2013,-0.110457,0.066879
2014,-0.021752,0.068312
2015,0.126844,0.101624


In [172]:
# Define the trading cost in basis points
trading_cost_bps = 5
trading_cost = trading_cost_bps / 10000

# Calculate the portfolio returns with trading cost
long_short_portfolio_with_cost = long_short_portfolio.copy()
long_short_portfolio_with_cost['Long'] = long_short_portfolio_with_cost['Long'] - trading_cost
long_short_portfolio_with_cost['Short'] = -long_short_portfolio_with_cost['Short'] - trading_cost
long_short_portfolio_with_cost['Portfolio'] = long_short_portfolio_with_cost['Long'] + long_short_portfolio_with_cost['Short']

# Calculate annual return and annualized return volatility with trading cost
annual_returns_with_cost = long_short_portfolio_with_cost['Portfolio'].groupby(long_short_portfolio_with_cost.index.year).sum()
annual_volatility_with_cost = long_short_portfolio_with_cost['Portfolio'].groupby(long_short_portfolio_with_cost.index.year).std() * np.sqrt(252)

# Combine the results into a DataFrame
annual_performance_with_cost = pd.DataFrame({
    'Annual Return with Cost': annual_returns_with_cost,
    'Annualized Volatility with Cost': annual_volatility_with_cost
})

# Compare the results
comparison = annual_performance.join(annual_performance_with_cost)
comparison

,Annual Return,Annualized Volatility,Annual Return with Cost,Annualized Volatility with Cost
2006,-0.126113,0.066564,-0.376113,0.066564
2007,-0.144599,0.101674,-0.395599,0.101674
2008,-0.308981,0.379326,-0.561981,0.379326
2009,0.062507,0.242195,-0.189493,0.242195
2010,0.064228,0.137949,-0.187772,0.137949
2011,-0.218834,0.199909,-0.470834,0.199909
2012,-0.10003,0.084154,-0.35003,0.084154
2013,-0.110457,0.066879,-0.362457,0.066879
2014,-0.021752,0.068312,-0.273752,0.068312
2015,0.126844,0.101624,-0.125156,0.101624
